In [81]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train', 'test', 'train.csv', 'sample_submission.csv']


In [82]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D,Conv2D
from keras.layers import Activation,Dropout,Flatten,Dense,MaxPooling2D,BatchNormalization
from keras.applications import ResNet50
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
import math


In [83]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications

In [84]:
X_train = "../input/train/train"
Y_train = "../input/test/test"

In [85]:
len(X_train)

20

In [86]:
train_data = pd.read_csv('../input/train.csv',low_memory=False)
train_data['has_cactus'].value_counts()
train_data.has_cactus = train_data.has_cactus.astype(str)

In [87]:
print("The number of rows in test set is %d"%(len(os.listdir('../input/test/test'))))
print("The number of rows in train set is %d"%(len(os.listdir('../input/train/train'))))

The number of rows in test set is 4000
The number of rows in train set is 17500


In [88]:
#datagen = ImageDataGenerator(rescale=1./255) 
batch_size = 32

In [89]:
train_datagen = ImageDataGenerator(rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,vertical_flip=False,
        fill_mode='nearest')

In [90]:
valid_datagen = ImageDataGenerator(rescale=1/.255)

In [91]:
train_generator=train_datagen.flow_from_dataframe(train_data[:15001],directory=X_train,x_col='id',y_col='has_cactus',
                                           class_mode='binary',batch_size=batch_size,target_size=(224,224))

valid_generator = valid_datagen.flow_from_dataframe(train_data[15000:],directory=X_train,x_col='id',y_col='has_cactus',
                                           class_mode='binary',batch_size=batch_size,target_size=(224,224))

Found 15001 validated image filenames belonging to 2 classes.
Found 2500 validated image filenames belonging to 2 classes.


In [92]:
train_generator.class_indices

{'0': 0, '1': 1}

In [93]:
from keras.applications import ResNet50

In [94]:
base_model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
print('Model Loaded')

base_model.trainable = False


Model Loaded


In [95]:
top_model = Sequential()
top_model.add(base_model)
top_model.add(Flatten())
top_model.add(Dense(1024))
top_model.add(Activation('relu'))
top_model.add(Dropout(0.4))

top_model.add(Dense(1))
top_model.add(Activation('sigmoid'))

model=top_model
sgd = optimizers.SGD(lr=1e-1,decay=1e-3,momentum=0.99,nesterov=True)
model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()




_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_6 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              102761472 
_________________________________________________________________
activation_305 (Activation)  (None, 1024)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 1025      
_________________________________________________________________
activation_306 (Activation)  (None, 1)                 0         
Total para

In [96]:
file_path="weights.best.hdf5"

checkpoint = ModelCheckpoint(file_path,monitor='acc', verbose=1, save_best_only=True, mode='max')

early = EarlyStopping(monitor="acc", mode="max", patience=15)

callbacks_list = [checkpoint, early] 

history = model.fit_generator(train_generator,
                    steps_per_epoch= 50,
                    epochs= 100 , verbose=2,
                    validation_data=valid_generator,
                    validation_steps=50,callbacks=callbacks_list)

Epoch 1/100
 - 34s - loss: 3.8932 - acc: 0.7500 - val_loss: 4.0255 - val_acc: 0.7475

Epoch 00001: acc improved from -inf to 0.75000, saving model to weights.best.hdf5
Epoch 2/100
 - 18s - loss: 3.9956 - acc: 0.7494 - val_loss: 4.0363 - val_acc: 0.7468

Epoch 00002: acc did not improve from 0.75000
Epoch 3/100
 - 20s - loss: 4.1649 - acc: 0.7388 - val_loss: 3.9258 - val_acc: 0.7538

Epoch 00003: acc did not improve from 0.75000
Epoch 4/100
 - 19s - loss: 3.8959 - acc: 0.7556 - val_loss: 4.0464 - val_acc: 0.7462

Epoch 00004: acc improved from 0.75000 to 0.75562, saving model to weights.best.hdf5
Epoch 5/100
 - 20s - loss: 3.9158 - acc: 0.7544 - val_loss: 3.8842 - val_acc: 0.7564

Epoch 00005: acc did not improve from 0.75562
Epoch 6/100
 - 19s - loss: 4.0055 - acc: 0.7488 - val_loss: 4.1450 - val_acc: 0.7400

Epoch 00006: acc did not improve from 0.75562
Epoch 7/100
 - 19s - loss: 4.0454 - acc: 0.7463 - val_loss: 3.9247 - val_acc: 0.7538

Epoch 00007: acc did not improve from 0.75562
E